In [1]:
import pandas as pd
import numpy as np

In [2]:
print(f"Pandas: {pd.__version__}")
print(f"NumPy: {np.__version__}")

Pandas: 2.3.3
NumPy: 2.3.4


In [39]:
import pandas as pd
import numpy as np

def analizar_dataframe(df):
    """
    Análisis eficiente de DataFrame usando operaciones vectorizadas.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame a analizar
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame con el análisis de cada columna
    """
    
    # Pre-calcular información común para todas las columnas
    total_filas = len(df)
    
    # Lista para almacenar resultados
    resultados = []
    
    for columna in df.columns:
        col_data = df[columna]
        
        # Tipo de dato (más eficiente con select_dtypes)
        es_numerica = pd.api.types.is_numeric_dtype(col_data)
        tipo = 'Numérica' if es_numerica else 'Alfanumérica'
        
        # Máscara de nulos (calcular una sola vez)
        mask_nulos = col_data.isna()
        cantidad_nulos = mask_nulos.sum()
        
        # Valores distintos (incluidos null) - muy eficiente
        valores_distintos = col_data.nunique(dropna=False)
        
        # Inicializar contadores
        cantidad_blancos = 0
        cantidad_ceros = 0
        cantidad_negativos = 0
        
        if es_numerica:
            # Operaciones vectorizadas para numéricos (muy rápido)
            col_no_nulos = col_data.dropna()
            if len(col_no_nulos) > 0:
                cantidad_ceros = (col_no_nulos == 0).sum()
                cantidad_negativos = (col_no_nulos < 0).sum()
        else:
            # Para alfanuméricos, usar operaciones de string vectorizadas
            # Más eficiente que apply
            if col_data.dtype == 'object':
                # Filtrar solo strings y contar blancos en una operación
                mask_strings = col_data.apply(lambda x: isinstance(x, str))
                if mask_strings.any():
                    cantidad_blancos = col_data[mask_strings].str.strip().eq('').sum()
        
        # 5 valores de ejemplo (optimizado)
        valores_unicos = col_data.dropna().unique()
        valores_ejemplo = valores_unicos[:5] if len(valores_unicos) > 0 else []
        valores_ejemplo_str = ', '.join(map(str, valores_ejemplo))
        
        # Agregar resultados
        resultados.append({
            'Columna': columna,
            'Tipo': tipo,
            'Total Valores': total_filas,
            'Nulos': int(cantidad_nulos),
            'Valores Distintos': int(valores_distintos),
            'Valores Blancos': int(cantidad_blancos),
            'Valores Cero': int(cantidad_ceros),
            'Valores Negativos': int(cantidad_negativos),
            'Ejemplos': valores_ejemplo_str
        })
    
    return pd.DataFrame(resultados)


def analizar_csv(ruta_archivo, separador=',', encoding='utf-8', nrows=None):
    """
    Lee y analiza un archivo CSV de forma eficiente.
    
    Parameters:
    -----------
    ruta_archivo : str
        Ruta del archivo CSV
    separador : str, default=','
        Separador del CSV
    encoding : str, default='utf-8'
        Codificación del archivo
    nrows : int, optional
        Número de filas a leer (útil para archivos grandes)
        
    Returns:
    --------
    tuple
        (DataFrame original, DataFrame de análisis)
    """
    
    print(f"Leyendo archivo: {ruta_archivo}")
    
    # Leer CSV con optimizaciones
    df = pd.read_csv(
        ruta_archivo, 
        sep=separador, 
        encoding=encoding,
        nrows=nrows,
        low_memory=False  # Más rápido para archivos grandes
    )
    
    print(f"✓ Archivo cargado: {df.shape[0]:,} filas, {df.shape[1]} columnas\n")
    
    print("Analizando columnas...")
    df_analisis = analizar_dataframe(df)
    print("✓ Análisis completado\n")
    
    return df, df_analisis


# ============================================
# VERSIÓN ULTRA OPTIMIZADA (para Big Data)
# ============================================

def analizar_csv_chunked(ruta_archivo, chunksize=10000, separador=',', encoding='utf-8'):
    """
    Analiza archivos CSV muy grandes por chunks (evita cargar todo en memoria).
    Ideal para datasets de millones de filas.
    
    Parameters:
    -----------
    ruta_archivo : str
        Ruta del archivo CSV
    chunksize : int, default=10000
        Número de filas por chunk
    separador : str, default=','
        Separador del CSV
    encoding : str, default='utf-8'
        Codificación del archivo
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame con el análisis de cada columna
    """
    
    print(f"Analizando archivo por chunks de {chunksize:,} filas...")
    
    # Diccionarios para acumular estadísticas
    stats = {}
    total_rows = 0
    
    # Leer por chunks
    for i, chunk in enumerate(pd.read_csv(ruta_archivo, sep=separador, 
                                          encoding=encoding, chunksize=chunksize)):
        
        total_rows += len(chunk)
        
        for columna in chunk.columns:
            if columna not in stats:
                # Inicializar estadísticas para nueva columna
                es_numerica = pd.api.types.is_numeric_dtype(chunk[columna])
                stats[columna] = {
                    'tipo': 'Numérica' if es_numerica else 'Alfanumérica',
                    'nulos': 0,
                    'valores_unicos': set(),
                    'blancos': 0,
                    'ceros': 0,
                    'negativos': 0,
                    'ejemplos': []
                }
            
            col_data = chunk[columna]
            col_stats = stats[columna]
            
            # Acumular nulos
            col_stats['nulos'] += col_data.isna().sum()
            
            # Acumular valores únicos (limitado para eficiencia)
            if len(col_stats['valores_unicos']) < 10000:
                col_stats['valores_unicos'].update(col_data.dropna().unique())
            
            # Ejemplos (solo del primer chunk)
            if i == 0 and len(col_stats['ejemplos']) < 5:
                ejemplos = col_data.dropna().unique()[:5].tolist()
                col_stats['ejemplos'].extend(ejemplos)
            
            # Estadísticas específicas por tipo
            if col_stats['tipo'] == 'Numérica':
                col_no_nulos = col_data.dropna()
                col_stats['ceros'] += (col_no_nulos == 0).sum()
                col_stats['negativos'] += (col_no_nulos < 0).sum()
            else:
                if col_data.dtype == 'object':
                    mask_strings = col_data.apply(lambda x: isinstance(x, str))
                    if mask_strings.any():
                        col_stats['blancos'] += col_data[mask_strings].str.strip().eq('').sum()
        
        if (i + 1) % 10 == 0:
            print(f"  Procesados {total_rows:,} filas...")
    
    print(f"✓ Análisis completado: {total_rows:,} filas totales\n")
    
    # Construir DataFrame de resultados
    resultados = []
    for columna, col_stats in stats.items():
        resultados.append({
            'Columna': columna,
            'Tipo': col_stats['tipo'],
            'Total Valores': total_rows,
            'Nulos': int(col_stats['nulos']),
            'Valores Distintos': len(col_stats['valores_unicos']),
            'Valores Blancos': int(col_stats['blancos']),
            'Valores Cero': int(col_stats['ceros']),
            'Valores Negativos': int(col_stats['negativos']),
            'Ejemplos': ', '.join(map(str, col_stats['ejemplos'][:5]))
        })
    
    return pd.DataFrame(resultados)


# ============================================
# EJEMPLO DE USO
# ============================================

if __name__ == "__main__":
    
    # ===== PARA ARCHIVOS NORMALES (< 1 GB) =====
    ruta_csv = 'C:\\Users\\fierr\\OneDrive\\Documents\\UDEMY\\forecastWallmark\\forecast\\ml_forecast_time_series\\data\\raw\\train.csv'
    df_original, df_analisis = analizar_csv(ruta_csv)
    
    print("=" * 100)
    print("ANÁLISIS DE COLUMNAS")
    print("=" * 100)
    print(df_analisis.to_string(index=False))
    
    # ===== PARA ARCHIVOS GRANDES (> 1 GB) =====
    df_analisis = analizar_csv_chunked(ruta_csv, chunksize=50000)
    display(df_analisis)
    
    # Guardar resultado
    df_analisis.to_csv('reports/analisis_columnas.csv', index=False)
    print("\n✓ Análisis guardado en: reports/analisis_columnas.csv")

Leyendo archivo: C:\Users\fierr\OneDrive\Documents\UDEMY\forecastWallmark\forecast\ml_forecast_time_series\data\raw\train.csv
✓ Archivo cargado: 913,000 filas, 4 columnas

Analizando columnas...
✓ Análisis completado

ANÁLISIS DE COLUMNAS
Columna         Tipo  Total Valores  Nulos  Valores Distintos  Valores Blancos  Valores Cero  Valores Negativos                                                   Ejemplos
   date Alfanumérica         913000      0               1826                0             0                  0 2013-01-01, 2013-01-02, 2013-01-03, 2013-01-04, 2013-01-05
  store     Numérica         913000      0                 10                0             0                  0                                              1, 2, 3, 4, 5
   item     Numérica         913000      0                 50                0             0                  0                                              1, 2, 3, 4, 5
  sales     Numérica         913000      0                213                

,Columna,Tipo,Total Valores,Nulos,Valores Distintos,Valores Blancos,Valores Cero,Valores Negativos,Ejemplos
0,date,Alfanumérica,913000,0,1826,0,0,0,"2013-01-01, 2013-01-02, 2013-01-03, 2013-01-04..."
1,store,Numérica,913000,0,10,0,0,0,"1, 2, 3, 4, 5"
2,item,Numérica,913000,0,50,0,0,0,"1, 2, 3"
3,sales,Numérica,913000,0,213,0,1,0,"13, 11, 14, 10, 12"


OSError: Cannot save file into a non-existent directory: 'reports'

In [ ]:
import pandas as pd
import numpy as np
from tabulate import tabulate

# ============================================
# ANÁLISIS SIMPLE DE SERIES DE TIEMPO
# ============================================

def analizar_series(df, col_fecha='date', col_store='store', col_item='item'):
    """
    Análisis rápido de series de tiempo por store e item.
    """
    
    print("="*80)
    print("ANÁLISIS DE SERIES DE TIEMPO")
    print("="*80 + "\n")
    
    # Convertir fecha si es necesario
    if not pd.api.types.is_datetime64_any_dtype(df[col_fecha]):
        df[col_fecha] = pd.to_datetime(df[col_fecha])
    
    print(f"📊 Total registros: {len(df):,}")
    print(f"🏪 Stores únicos: {df[col_store].nunique():,}")
    print(f"📦 Items únicos: {df[col_item].nunique():,}")
    
    # Análisis por (store, item)
    analisis = df.groupby([col_store, col_item]).agg(
        registros=pd.NamedAgg(column=col_fecha, aggfunc='count'),
        fecha_min=pd.NamedAgg(column=col_fecha, aggfunc='min'),
        fecha_max=pd.NamedAgg(column=col_fecha, aggfunc='max')
    ).reset_index()
    
    # Calcular días
    analisis['dias_totales'] = (analisis['fecha_max'] - analisis['fecha_min']).dt.days + 1
    analisis['dias_esperados'] = analisis['dias_totales']
    analisis['completitud_pct'] = (analisis['registros'] / analisis['dias_esperados'] * 100).round(2)
    analisis['registros_faltantes'] = analisis['dias_esperados'] - analisis['registros']
    
    print(f"🔢 Combinaciones (store, item): {len(analisis):,}\n")
    
    return analisis


def analizar_por_deciles(analisis_df):
    """
    Agrupa las series por deciles según el número de registros.
    """
    
    # Calcular deciles
    try:
        analisis_df['decil'] = pd.qcut(
            analisis_df['registros'], 
            q=10, 
            labels=False,
            duplicates='drop'
        )
        # Convertir a etiquetas D1, D2, etc.
        analisis_df['decil'] = 'D' + (analisis_df['decil'] + 1).astype(str)
    except Exception:
        # Si no se pueden calcular 10 deciles, usar percentiles
        analisis_df['decil'] = pd.cut(
            analisis_df['registros'],
            bins=10,
            labels=['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10']
        )
    
    # Estadísticas por decil
    deciles = analisis_df.groupby('decil', observed=True).agg({
        'store': 'count',
        'registros': ['min', 'max', 'mean'],
        'completitud_pct': 'mean',
        'registros_faltantes': ['mean', 'sum']
    }).round(2)
    
    deciles.columns = ['num_series', 'registros_min', 'registros_max', 
                      'registros_prom', 'completitud_prom', 
                      'faltantes_prom', 'faltantes_total']
    
    return deciles, analisis_df


def mostrar_resultados(analisis_df, deciles_df):
    """
    Muestra los resultados del análisis.
    """
    
    print("="*80)
    print("RESUMEN ESTADÍSTICO")
    print("="*80 + "\n")
    
    resumen_data = {
        'Métrica': [
            'Combinaciones (store, item)',
            'Registros promedio',
            'Registros mínimo',
            'Registros máximo',
            'Completitud promedio',
            'Series 100% completas',
            'Series con gaps'
        ],
        'Valor': [
            f"{len(analisis_df):,}",
            f"{analisis_df['registros'].mean():.0f}",
            f"{analisis_df['registros'].min():,}",
            f"{analisis_df['registros'].max():,}",
            f"{analisis_df['completitud_pct'].mean():.1f}%",
            f"{(analisis_df['completitud_pct'] == 100).sum():,} ({(analisis_df['completitud_pct'] == 100).sum()/len(analisis_df)*100:.1f}%)",
            f"{(analisis_df['registros_faltantes'] > 0).sum():,} ({(analisis_df['registros_faltantes'] > 0).sum()/len(analisis_df)*100:.1f}%)"
        ]
    }
    
    print(tabulate(pd.DataFrame(resumen_data), headers='keys', tablefmt='grid', showindex=False))
    
    print("\n" + "="*80)
    print("ANÁLISIS POR DECILES (según número de registros)")
    print("="*80 + "\n")
    print(tabulate(deciles_df, headers='keys', tablefmt='grid'))
    
    # Distribución de completitud
    print("\n" + "="*80)
    print("DISTRIBUCIÓN DE COMPLETITUD")
    print("="*80 + "\n")
    
    bins_completitud = [0, 50, 70, 80, 90, 95, 99, 100]
    analisis_df['rango_completitud'] = pd.cut(
        analisis_df['completitud_pct'], 
        bins=bins_completitud,
        labels=['0-50%', '50-70%', '70-80%', '80-90%', '90-95%', '95-99%', '99-100%']
    )
    
    dist_completitud = analisis_df['rango_completitud'].value_counts().sort_index()
    dist_completitud_pct = (dist_completitud / len(analisis_df) * 100).round(2)
    
    completitud_tabla = pd.DataFrame({
        'Rango Completitud': dist_completitud.index,
        'Num Series': dist_completitud.values,
        '% del Total': dist_completitud_pct.values
    })
    
    print(tabulate(completitud_tabla, headers='keys', tablefmt='grid', showindex=False))


# ============================================
# FUNCIÓN PRINCIPAL - USO RÁPIDO
# ============================================

def analisis_completo(df, exportar=True):
    """
    Ejecuta análisis completo y exporta resultados.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame con columnas: date, store, item
    exportar : bool
        Si True, exporta a CSV
    
    Returns:
    --------
    tuple
        (analisis_df, deciles_df)
    """
    
    # 1. Análisis por serie
    analisis_df = analizar_series(df)
    
    # 2. Análisis por deciles
    deciles_df, analisis_df = analizar_por_deciles(analisis_df)
    
    # 3. Mostrar resultados
    mostrar_resultados(analisis_df, deciles_df)
    
    # 4. Exportar
    if exportar:
        print("\n" + "="*80)
        print("EXPORTANDO RESULTADOS")
        print("="*80 + "\n")
        
        analisis_df.to_csv('analisis_series_completo.csv', index=False)
        print("✓ analisis_series_completo.csv")
        
        deciles_df.to_csv('analisis_deciles.csv')
        print("✓ analisis_deciles.csv")
        
        # Exportar series con problemas
        series_problemas = analisis_df[analisis_df['completitud_pct'] < 90]
        if len(series_problemas) > 0:
            series_problemas.to_csv('series_incompletas.csv', index=False)
            print("✓ series_incompletas.csv")
    
    return analisis_df, deciles_df


# ============================================
# EJEMPLO DE USO CON TU DATASET
# ============================================

if __name__ == "__main__":
    
    # Ruta de tu archivo
    ruta_archivo = r'C:\Users\fierr\OneDrive\Documents\UDEMY\forecastWallmark\forecast\ml_forecast_time_series\data\raw\train.csv'
    
    # Schema de tipos
    raw_schema = {
        'store': 'Int64',
        'item': 'Int64',
        'sales': 'Int64'
    }
    
    # Schema de fechas
    date_schema = {
        'date': '%Y-%m-%d'
    }
    
    # Leer CSV
    print("📂 Leyendo CSV...\n")
    
    # Opción 1: Lectura simple
    df = pd.read_csv(ruta_archivo, dtype=raw_schema, parse_dates=['date'])
    
    # O usando tu función personalizada:
    # from tu_funcion_mejorada import leer_csv
    # df = leer_csv(ruta_archivo, df_schema=raw_schema, date_schema=date_schema)
    
    print(f"✓ Datos cargados: {df.shape[0]:,} registros\n")
    
    # Ejecutar análisis completo
    analisis_df, deciles_df = analisis_completo(df, exportar=True)
    
    # ========================================
    # CONSULTAS ADICIONALES
    # ========================================
    
    print("\n" + "="*80)
    print("CONSULTAS ADICIONALES")
    print("="*80 + "\n")
    
    # Top 10 series con más registros
    print("📊 TOP 10 SERIES CON MÁS REGISTROS:")
    top_10 = analisis_df.nlargest(10, 'registros')[['store', 'item', 'registros', 'completitud_pct']]
    print(tabulate(top_10, headers='keys', tablefmt='grid', showindex=False))
    
    print("\n📊 TOP 10 SERIES CON MENOS REGISTROS:")
    bottom_10 = analisis_df.nsmallest(10, 'registros')[['store', 'item', 'registros', 'completitud_pct']]
    print(tabulate(bottom_10, headers='keys', tablefmt='grid', showindex=False))
    
    # Series por decil
    print("\n📊 NÚMERO DE SERIES POR DECIL:")
    series_por_decil = analisis_df['decil'].value_counts().sort_index()
    print(tabulate(series_por_decil.reset_index(), headers=['Decil', 'Num Series'], 
                  tablefmt='grid', showindex=False))
    
    # Ejemplo: filtrar serie específica
    print("\n📊 EJEMPLO: Serie Store=1, Item=1:")
    serie_ejemplo = analisis_df[(analisis_df['store'] == 1) & (analisis_df['item'] == 1)]
    if len(serie_ejemplo) > 0:
        print(tabulate(serie_ejemplo[['store', 'item', 'registros', 'fecha_min', 
                                     'fecha_max', 'dias_totales', 'completitud_pct']], 
                      headers='keys', tablefmt='grid', showindex=False))
    
    # Series por rango de completitud
    print("\n📊 SERIES POR RANGO DE COMPLETITUD:")
    completitud_rangos = analisis_df.groupby('rango_completitud', observed=True).size()
    print(tabulate(completitud_rangos.reset_index(), headers=['Rango', 'Cantidad'], 
                  tablefmt='grid', showindex=False))
    
    # Estadísticas por decil
    print("\n📊 ESTADÍSTICAS POR DECIL:")
    for decil in ['D1', 'D5', 'D10']:
        if decil in analisis_df['decil'].values:
            series_decil = analisis_df[analisis_df['decil'] == decil]
            print(f"\n{decil}:")
            print(f"  - Series: {len(series_decil)}")
            print(f"  - Registros promedio: {series_decil['registros'].mean():.0f}")
            print(f"  - Registros min-max: {series_decil['registros'].min()}-{series_decil['registros'].max()}")
            print(f"  - Completitud promedio: {series_decil['completitud_pct'].mean():.1f}%")

# COD

In [40]:
def leer_csv(ruta, separador=',', encoding='utf-8', df_schema={}, date_schema={}):
    df = pd.read_csv(
        ruta,
        sep = separador,                          # Separador
        encoding = encoding,                 # Codificación
        dtype = df_schema,                 # ← AQUÍ VA EL DICCIONARIO
        #parse_dates = date,                 # Columnas de fecha
        #na_values=valores_nulos,          # Qué considerar como nulo
        low_memory = False                  # Más rápido para archivos grandes
    )

    #convertimos lo que sea fecha
    if date_schema:
        for col_fecha, formato in date_schema.items():
            df[col_fecha] = pd.to_datetime(df[col_fecha], format=formato, errors='coerce')
            #print(f"   ✓ {col_fecha} → datetime ({formato})")

    return df

def analizar_csv(df):
    # Pre-calcular información común para todas las columnas
    total_filas = len(df)
    
    # Lista para almacenar resultados
    resultados = []
    
    for columna in df.columns:
        col_data = df[columna]
        
        # Tipo de dato (más eficiente con select_dtypes)
        es_numerica = pd.api.types.is_numeric_dtype(col_data)
        tipo = 'Numérica' if es_numerica else 'Alfanumérica'
        
        # Máscara de nulos (calcular una sola vez)
        mask_nulos = col_data.isna()
        cantidad_nulos = mask_nulos.sum()
        
        # Valores distintos (incluidos null) - muy eficiente
        valores_distintos = col_data.nunique(dropna=False)
        
        # Inicializar contadores
        cantidad_blancos = 0
        cantidad_ceros = 0
        cantidad_negativos = 0
        
        if es_numerica:
            # Operaciones vectorizadas para numéricos (muy rápido)
            col_no_nulos = col_data.dropna()
            if len(col_no_nulos) > 0:
                cantidad_ceros = (col_no_nulos == 0).sum()
                cantidad_negativos = (col_no_nulos < 0).sum()
        else:
            # Para alfanuméricos, usar operaciones de string vectorizadas
            # Más eficiente que apply
            if col_data.dtype == 'object':
                # Filtrar solo strings y contar blancos en una operación
                mask_strings = col_data.apply(lambda x: isinstance(x, str))
                if mask_strings.any():
                    cantidad_blancos = col_data[mask_strings].str.strip().eq('').sum()
        
        # 5 valores de ejemplo (optimizado)
        valores_unicos = col_data.dropna().unique()
        valores_ejemplo = valores_unicos[:5] if len(valores_unicos) > 0 else []
        valores_ejemplo_str = ', '.join(map(str, valores_ejemplo))
        
        # Agregar resultados
        resultados.append({
            'Columna': columna,
            'Tipo': tipo,
            'Total Valores': total_filas,
            'Nulos': int(cantidad_nulos),
            'Valores Distintos': int(valores_distintos),
            'Valores Blancos': int(cantidad_blancos),
            'Valores Cero': int(cantidad_ceros),
            'Valores Negativos': int(cantidad_negativos),
            'Ejemplos': valores_ejemplo_str
        })
    
    return pd.DataFrame(resultados)

## Identificadores:

- store: Código de la tienda
- item: Código del producto

Fechas de actividad:

- local_start_date: Primera fecha en que este producto se vendió en esta tienda
- local_end_date: Última fecha en que este producto se vendió en esta tienda
- general_start_date: Fecha más antigua con ventas en todo el sistema
- general_end_date: Fecha más reciente con ventas en todo el sistema

Cobertura de datos (período propio):

- local_periods_total: Total de días entre la primera y última venta del producto en la tienda
- local_periods_with_data: Días en que hubo al menos una transacción registrada
- local_periods_missing: Días sin ningún registro entre primera y última venta (gaps en los datos)
- local_null_sales_count: Días donde hubo registros pero todas las ventas aparecen sin valor (datos incompletos)

Cobertura de datos (período extendido):

- extended_periods_total: Días desde la primera venta del producto hasta la fecha más reciente del sistema
- extended_periods_with_data: Días con transacciones en ese período extendido
- extended_periods_missing: Días sin datos desde que el producto comenzó hasta hoy
- extended_null_sales_count: Días con datos incompletos en ese período extendido

In [41]:
def analizar_series_tiempo(df, granularidad, col_fecha, cols_id, col_valor):
    """ granularidad:
    'D' - Día (daily)
    'W' - Semana (weekly)
    'M' - Mes (month end)
    'MS' - Mes (month start)
    'Q' - Trimestre (quarter end)
    'QS' - Trimestre (quarter start)
    'Y' o 'A' - Año (year end)
    'YS' o 'AS' - Año (year start)

    Otros útiles:

    'H' - Hora (hourly)
    'T' o 'min' - Minuto (minute)
    'S' - Segundo (second)
    'B' - Día hábil (business day)
    'W-MON', 'W-TUE', etc. - Semana comenzando en día específico
    """
    print("F1")
    # Convertir fecha a datetime y filtrar nulls en fecha
    df[col_fecha] = pd.to_datetime(df[col_fecha])
    df_clean = df[df[col_fecha].notna()].copy()
    
    # Fechas globales
    general_start_date = df_clean[col_fecha].min()
    general_end_date = df_clean[col_fecha].max()
    print(general_start_date, '-', general_end_date)
    
    # Agrupar por cols_id
    resultado = df_clean.groupby(cols_id).agg({
        col_fecha: ['min', 'max', 'nunique']
    }).reset_index()
    
    # Renombrar columnas
    # local_periods_with_data considera aun peridos con nulls en sales
    resultado.columns = cols_id + ['local_start_date', 'local_end_date', 'local_periods_with_data']
    
    
    print("F2")
    # Contar períodos donde TODOS los sales son null (optimizado)
    df_temp = df_clean.copy()
    df_temp['is_null'] = df_temp[col_valor].isna()
    
    periods_all_null = df_temp.groupby(cols_id + [col_fecha])['is_null'].all().reset_index(name='all_null')
    
    print("F2.1")
    periods_all_null = periods_all_null.groupby(cols_id)['all_null'].sum().reset_index()
    periods_all_null.columns = cols_id + ['local_null_sales_count']
    
    print("F2.2")
    resultado = resultado.merge(periods_all_null, on=cols_id, how='left')
    # local_null_sales_count tendrá la cantidad de periodos en que todos los sales son null
    resultado['local_null_sales_count'] = resultado['local_null_sales_count'].fillna(0).astype(int)
    
    print("F2.3")
    # Agregar fechas globales
    resultado['general_start_date'] = general_start_date
    resultado['general_end_date'] = general_end_date
    
    print("F3")
    # Períodos totales y faltantes (rango local)
    resultado['local_periods_total'] = resultado.apply(
        lambda row: len(pd.date_range(row['local_start_date'], row['local_end_date'], freq=granularidad.upper())),
        axis=1
    )
    resultado['local_periods_missing'] = resultado['local_periods_total'] - resultado['local_periods_with_data']
    
    # Períodos totales y faltantes (rango extendido: local_start a general_end)
    resultado['extended_periods_total'] = resultado.apply(
        lambda row: len(pd.date_range(row['local_start_date'], general_end_date, freq=granularidad.upper())),
        axis=1
    )
    
    print("F4")
    # Los períodos con datos son los mismos porque cada grupo solo tiene datos hasta su local_end
    resultado['extended_periods_with_data'] = resultado['local_periods_with_data']
    resultado['extended_periods_missing'] = resultado['extended_periods_total'] - resultado['extended_periods_with_data']
    
    # Los períodos con todos-nulls también son los mismos (solo existen hasta local_end)
    resultado['extended_null_sales_count'] = resultado['local_null_sales_count']
    
    print("F5")
    display(resultado.head())
    return resultado


    
    #cantidad de 

    #cuantas series 

if __name__ == "__main__":
    ruta_archivo = 'C:\\Users\\fierr\\OneDrive\\Documents\\UDEMY\\forecastWallmark\\forecast\\ml_forecast_time_series\\data\\raw\\train.csv'
    separador=','
    encoding='utf-8'

    raw_schema={
        'date': 'string',
        'store': 'Int64',        # Con mayúscula acepta nulos
        'item': 'Int64',
        'sales': 'Int64'
    }

    date_schema={'date': '%Y-%m-%d'}

    df = leer_csv(ruta_archivo,separador,encoding,raw_schema,date_schema)
    df_analisis= analizar_csv(df.copy())
    display(df_analisis)
    
    df_analisis_series = analizar_series_tiempo(df.copy(), 'd','date', ['store','item'], 'sales')
    #Validar ingresando unas ingreso de un registro con 4 valores

,Columna,Tipo,Total Valores,Nulos,Valores Distintos,Valores Blancos,Valores Cero,Valores Negativos,Ejemplos
0,date,Alfanumérica,913000,0,1826,0,0,0,"2013-01-01 00:00:00, 2013-01-02 00:00:00, 2013..."
1,store,Numérica,913000,0,10,0,0,0,"1, 2, 3, 4, 5"
2,item,Numérica,913000,0,50,0,0,0,"1, 2, 3, 4, 5"
3,sales,Numérica,913000,0,213,0,1,0,"13, 11, 14, 10, 12"


F1
2013-01-01 00:00:00 - 2017-12-31 00:00:00
F2
F2.1
F2.2
F2.3
F3
F4
F5


,store,item,local_start_date,local_end_date,local_periods_with_data,local_null_sales_count,general_start_date,general_end_date,local_periods_total,local_periods_missing,extended_periods_total,extended_periods_with_data,extended_periods_missing,extended_null_sales_count
0,1,1,2013-01-01,2017-12-31,1826,0,2013-01-01,2017-12-31,1826,0,1826,1826,0,0
1,1,2,2013-01-01,2017-12-31,1826,0,2013-01-01,2017-12-31,1826,0,1826,1826,0,0
2,1,3,2013-01-01,2017-12-31,1826,0,2013-01-01,2017-12-31,1826,0,1826,1826,0,0
3,1,4,2013-01-01,2017-12-31,1826,0,2013-01-01,2017-12-31,1826,0,1826,1826,0,0
4,1,5,2013-01-01,2017-12-31,1826,0,2013-01-01,2017-12-31,1826,0,1826,1826,0,0


In [42]:
df_analisis_series.describe()

,store,item,local_start_date,local_end_date,local_periods_with_data,local_null_sales_count,general_start_date,general_end_date,local_periods_total,local_periods_missing,extended_periods_total,extended_periods_with_data,extended_periods_missing,extended_null_sales_count
count,500.0,500.0,500,500,500.0,500.0,500,500,500.0,500.0,500.0,500.0,500.0,500.0
mean,5.5,25.5,2013-01-01 00:00:00,2017-12-31 00:00:00,1826.0,0.0,2013-01-01 00:00:00,2017-12-31 00:00:00,1826.0,0.0,1826.0,1826.0,0.0,0.0
min,1.0,1.0,2013-01-01 00:00:00,2017-12-31 00:00:00,1826.0,0.0,2013-01-01 00:00:00,2017-12-31 00:00:00,1826.0,0.0,1826.0,1826.0,0.0,0.0
25%,3.0,13.0,2013-01-01 00:00:00,2017-12-31 00:00:00,1826.0,0.0,2013-01-01 00:00:00,2017-12-31 00:00:00,1826.0,0.0,1826.0,1826.0,0.0,0.0
50%,5.5,25.5,2013-01-01 00:00:00,2017-12-31 00:00:00,1826.0,0.0,2013-01-01 00:00:00,2017-12-31 00:00:00,1826.0,0.0,1826.0,1826.0,0.0,0.0
75%,8.0,38.0,2013-01-01 00:00:00,2017-12-31 00:00:00,1826.0,0.0,2013-01-01 00:00:00,2017-12-31 00:00:00,1826.0,0.0,1826.0,1826.0,0.0,0.0
max,10.0,50.0,2013-01-01 00:00:00,2017-12-31 00:00:00,1826.0,0.0,2013-01-01 00:00:00,2017-12-31 00:00:00,1826.0,0.0,1826.0,1826.0,0.0,0.0
std,2.875158,14.445322,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 913000 entries, 0 to 912999
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   date    913000 non-null  datetime64[ns]
 1   store   913000 non-null  Int64         
 2   item    913000 non-null  Int64         
 3   sales   913000 non-null  Int64         
dtypes: Int64(3), datetime64[ns](1)
memory usage: 30.5 MB


In [ ]:
leer csv
evaluar analisis de columnas
EDA
    outliers
    distribucion
    graficos
procesar datos
metodos de prediccion 